<a href="https://colab.research.google.com/github/05-515/stepik_deep_learning/blob/main/%5Bseminar%5Dtext_preprocessing_and_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предобработка текста

## Часть 1

### Токенизация

In [2]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
data = "All work and no play makes jack a dull boy, all work and no play"
#приводим строку к нижнему регистру и проводим токенизацию по словам
#тоукенизируем по словам, знакам препинания.в основе каждого токенайзера алгоритм
tokens = word_tokenize(data.lower())
print(tokens)

['all', 'work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', ',', 'all', 'work', 'and', 'no', 'play']


In [4]:
print(sent_tokenize("I was going home when she rung. It was a surprise."))

['I was going home when she rung.', 'It was a surprise.']


[<img src="https://raw.githubusercontent.com/natasha/natasha-logos/master/natasha.svg">](https://github.com/natasha/natasha)

[Razdel](https://natasha.github.io/razdel/)
Удобная библиотека для работы с русским текстом гораздо лучше word_tokenizer

In [5]:
!pip install -q razdel

In [6]:
from razdel import tokenize, sentenize
text = 'Кружка-термос на 0.5л (50/64 см³, 516;...)'
list(tokenize(text))

[Substring(0, 13, 'Кружка-термос'),
 Substring(14, 16, 'на'),
 Substring(17, 20, '0.5'),
 Substring(20, 21, 'л'),
 Substring(22, 23, '('),
 Substring(23, 28, '50/64'),
 Substring(29, 32, 'см³'),
 Substring(32, 33, ','),
 Substring(34, 37, '516'),
 Substring(37, 38, ';'),
 Substring(38, 41, '...'),
 Substring(41, 42, ')')]

#### Регулярные выражения

Исчерпывающий пост https://habr.com/ru/post/349860/

Внутри каждого токенайзера лежит алгоритм. по которому делим наш токен. находим некий шаблон и по этим шаблонам делим. Коснемся этих выражений. Регулярные выражения

In [7]:
import re
word = 'supercalifragilisticexpialidocious'
# функция findall. находим по заданному шаблону
re.findall('[abc]|up|super', word) 
# [abc] выводит любой символ из них 
# | - или 
# идем слева направо/ порядок имеет значение. поэтому выводим super, up уже не видим, отработали эти символы

['super', 'c', 'a', 'a', 'c', 'a', 'c']

In [8]:
re.findall('\d{1,2}', 'These are some numbers: 49 and 432')
#\d - ищем цифры 
# 1,2 означает однозначные или 2 значные числа
# почему 2 выводит. 43 нашли и вывели. потом 2 обрабатываем как однозначное число с помощью 1
# 3 добавим и будем еще находить 3 значные числа
re.findall('\d{1,3}', 'These are some numbers: 49 and 432432')


['49', '432', '432']

In [9]:
re.sub('[,\.?!]','','How, to? split. text!')
# в скобочках стоят элементы, которые удаляем при нахождении. замена на пустоту

'How to split text'

In [10]:
re.sub('[^A-z]',' ','I 123 can 45 play 67 football').split()
# ^ говорит о том,что удаляем все, кроме 
# A-z весь диапазон
#некий свой токейнайзер сделали

['I', 'can', 'play', 'football']

### Удаление неинформативных слов

хотим уменишить размер нашего словаря. должны поработать над тем ,как важно наше слово. неинформативные удалим

#### N-граммы


<img src="https://res.cloudinary.com/practicaldev/image/fetch/s--466CQV1q--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_66%2Cw_880/https://thepracticaldev.s3.amazonaws.com/i/78nf1vryed8h1tz05fim.gif" height=400>

это последовательность n-токенов
униграммы находят популярные токены
биграмы находят уже популярные взаимодействия токенов  
помогают выделить основные черты, шаблоны нашего текста

In [11]:
unigram = list(nltk.ngrams(tokens, 1))
bigram = list(nltk.ngrams(tokens, 2))
print(unigram[:5])
print(bigram[:5])

[('all',), ('work',), ('and',), ('no',), ('play',)]
[('all', 'work'), ('work', 'and'), ('and', 'no'), ('no', 'play'), ('play', 'makes')]


In [12]:
from nltk import FreqDist
#посмотреть как часто встречаются в тексте токены, какие слова частотные
#реддкие слова могуь быть нерелевантны для нас, с оишбкой, например. их можно удалять
print('Популярные униграммы: ', FreqDist(unigram).most_common(5))
print('Популярные биграммы: ', FreqDist(bigram).most_common(5))

Популярные униграммы:  [(('all',), 2), (('work',), 2), (('and',), 2), (('no',), 2), (('play',), 2)]
Популярные биграммы:  [(('all', 'work'), 2), (('work', 'and'), 2), (('and', 'no'), 2), (('no', 'play'), 2), (('play', 'makes'), 1)]


#### Стоп-слова
несут некую грамматическую роль, говорят нам о некоемом грамматическом значении нашего слова
местоимения, главголы связки для примера. 

In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
stopWords = set(stopwords.words('english'))
#можно поменять на russian
print(stopWords)

{'was', 'very', 'can', "should've", "won't", 'her', 'between', "don't", 'more', 'they', 'not', 'its', 'weren', 'off', 'than', "you've", 'if', 'for', 'nor', 't', 'over', 'hadn', 'yourself', 'yours', "mightn't", 'it', 'd', 'herself', 'ourselves', "wouldn't", 'each', 'ours', 'your', 've', 'aren', 'shouldn', 'our', 'why', 'have', 'no', 'to', "weren't", 'then', 'shan', 'them', "it's", "haven't", 'she', 'how', 'into', 'up', 'below', 'again', 'didn', 'this', 'am', 'couldn', 'those', 'where', "aren't", 'same', "you'd", 'itself', 'yourselves', 'all', "you're", 'ain', 'wasn', 'should', "wasn't", 'about', "isn't", 'whom', 'their', "she's", "you'll", 'does', 'from', 'when', "couldn't", 'll', 'until', 'in', 'do', 'with', 'y', 'an', 'needn', 'is', 'further', 'we', 'or', 'are', 'after', 'as', 's', 'and', 'that', 'any', 'had', 'by', 'mustn', 'own', 're', 'o', "shan't", 'above', 'did', 'isn', 'but', "needn't", 'few', 'his', 'of', 'him', 'both', "mustn't", "that'll", "doesn't", 'at', 'wouldn', 'hasn', '

In [15]:
print([word for word in tokens if word not in stopWords])

['work', 'play', 'makes', 'jack', 'dull', 'boy', ',', 'work', 'play']


In [16]:
# можем убрать из текста пунктуацию
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


#### Стемминг vs Лемматизация
* ‘Caring’ -> Лемматизация -> ‘Care’ более сложный процесс, переводим слово  к начальной форме
* ‘Caring’ -> Стемминг -> ‘Car’ нахождение основы слова

у слова может быть несколько форм. род и падеж у существительного меняется, но значение остается тем же. мы стремимся уменьшить наш словарь. хотим сделать преобразование слов к их лемме


### Стемминг
* процесс нахождения основы слова для заданного исходного слова

In [17]:
from nltk.stem import PorterStemmer, SnowballStemmer
#porter более старая функция, snow современнАЯ, усовершенстованная
words = ["game", "gaming", "gamed", "games", "compacted"]
words_ru = ['корова', 'мальчики', 'мужчины', 'столом', 'убежала']

In [18]:
ps = PorterStemmer()
list(map(ps.stem, words))

['game', 'game', 'game', 'game', 'compact']

In [19]:
ss = SnowballStemmer(language='russian')
list(map(ss.stem, words_ru))

['коров', 'мальчик', 'мужчин', 'стол', 'убежа']

### Лемматизация
* процесс приведения словоформы к лемме — её нормальной (словарной) форме

In [20]:
raw = """DENNIS: Listen, strange women lying in ponds distributing swords
is no basis for a system of government.  Supreme executive power derives from
a mandate from the masses, not from some farcical aquatic ceremony."""

raw_ru = """Не существует научных доказательств в пользу эффективности НЛП, оно
признано псевдонаукой. Систематические обзоры указывают, что НЛП основано на
устаревших представлениях об устройстве мозга, несовместимо с современной
неврологией и содержит ряд фактических ошибок."""

In [21]:
!pip install -q pymorphy2
# флаг -q не было лишних логов

In [22]:
# 1
import pymorphy2
#ru based библиотека. смотрим на основу слова, никак не на контекст
morph = pymorphy2.MorphAnalyzer()
pymorphy_results = list(map(lambda x: morph.parse(x), raw_ru.split(' '))) # разбиваем предложение по словам и отдельно парсим
print(' '.join([res[0].normal_form for res in pymorphy_results]))

не существовать научный доказательство в польза эффективность нлп, оно
признать псевдонаукой. систематический обзор указывают, что нлп основать на
устаревший представление о устройство мозга, несовместимый с современной
неврология и содержать ряд фактический ошибок.


In [23]:
pymorphy_results[2]
# у каждой основы есть своя вероятность

[Parse(word='научных', tag=OpencorporaTag('ADJF,Qual plur,gent'), normal_form='научный', score=0.774193, methods_stack=((DictionaryAnalyzer(), 'научных', 12, 21),)),
 Parse(word='научных', tag=OpencorporaTag('ADJF,Qual plur,loct'), normal_form='научный', score=0.209677, methods_stack=((DictionaryAnalyzer(), 'научных', 12, 26),)),
 Parse(word='научных', tag=OpencorporaTag('ADJF,Qual anim,plur,accs'), normal_form='научный', score=0.016129, methods_stack=((DictionaryAnalyzer(), 'научных', 12, 23),))]

In [24]:
# 2 библиотека работает контекстно, нейросетевой подход. в открытом доступе только с английским\разные. нет русского 
# создаем объект, закидываем наш текст, выделяем начальные формы
!pip install -U -q spacy
!python -m -q spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
spacy_results = nlp(raw)
print(' '.join([token.lemma_ for token in spacy_results]))

/home/codespace/.python/current/bin/python: No module named -q
DENNIS : listen , strange woman lie in pond distribute sword 
 be no basis for a system of government .   Supreme executive power derive from 
 a mandate from the masse , not from some farcical aquatic ceremony .


[Сравнение PyMorphy2 и PyMystem3](https://habr.com/ru/post/503420/)

эти библиотеки работают с русским, они role-based, ОНИ НЕ КОНТЕКСТНЫЕ
pymystem долго иницилизируется. с каждым предложением отдельно. если будет сразу бужет работать со твсем текстом, его объединим, то так быстрее и даже pymorphy2

### Part-of-Speech

некая предобработка текста
сделали:
уменьшили словарь, использзуем слово как признак нашего текста, но можем такжеь использовать признаки наших слов как признак текста.

In [25]:
# 1 кроме тега части речи,есть теги части слова
[(res[0].normal_form, res[0].tag) for res in pymorphy_results[:9]]

[('не', OpencorporaTag('PRCL')),
 ('существовать', OpencorporaTag('VERB,impf,intr sing,3per,pres,indc')),
 ('научный', OpencorporaTag('ADJF,Qual plur,gent')),
 ('доказательство', OpencorporaTag('NOUN,inan,neut plur,gent')),
 ('в', OpencorporaTag('PREP')),
 ('польза', OpencorporaTag('NOUN,inan,femn sing,accs')),
 ('эффективность', OpencorporaTag('NOUN,inan,femn sing,gent')),
 ('нлп,', OpencorporaTag('UNKN')),
 ('оно\nпризнать', OpencorporaTag('PRTS,perf,past,pssv neut,sing'))]

In [26]:
# 2 дописываем часть речи
[(token.lemma_, token.pos_) for token in spacy_results[:7]]

[('DENNIS', 'PROPN'),
 (':', 'PUNCT'),
 ('listen', 'VERB'),
 (',', 'PUNCT'),
 ('strange', 'ADJ'),
 ('woman', 'NOUN'),
 ('lie', 'VERB')]

In [27]:
#ХОРОШАЯ БИБлиотек, которая работает на основе реккурентных нейронных сетей, хорошо работает с русским языком
!pip install -q rnnmorph

In [28]:
# 3
#!pip install -q tensorflow
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")
#rnnmorph_result = predictor.predict(raw_ru.split(' '))
#[(token.normal_form, token.pos, token.tag) for token in rnnmorph_result[:7]]

2024-05-03 08:54:36.188658: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-03 08:54:39.407736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TypeError: Could not locate class 'Functional'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'class_name': 'Functional', 'config': {'name': 'model_1', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, None, 56], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'grammemes'}, 'name': 'grammemes', 'inbound_nodes': []}, {'class_name': 'Dropout', 'config': {'name': 'dropout_1', 'trainable': True, 'dtype': 'float32', 'rate': 0.2, 'noise_shape': None, 'seed': None}, 'name': 'dropout_1', 'inbound_nodes': [[['grammemes', 0, 0, {}]]]}, {'class_name': 'InputLayer', 'config': {'batch_input_shape': [None, None, 32], 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'chars'}, 'name': 'chars', 'inbound_nodes': []}, {'class_name': 'Dense', 'config': {'name': 'dense_1', 'trainable': True, 'dtype': 'float32', 'units': 30, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {

### Named entities recognition

пытаемся найти именнованные сущности. тоже являются признаки текста. могут не идти подряд. задача в том, что мы ищем конкретно именнованные сущности. 

In [29]:
# используем библиотекy spacy
# метод lower понижения к нижнему регистру мешает, важден регистр в таких задач приска именнованной сущности
doc = nlp('Apple is looking at buying U.K. startup for $1 billion')

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


С каждым методом нужно работать отделно. всегда нужно пробовать улучшить качество модели

## Часть 2

### Задача классификации

#### 20 newsgroups
Датасет с 18000 новостей, сгруппированных по 20 темам.

In [30]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [31]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [32]:
newsgroups_train.filenames.shape

(11314,)

#### Рассмотрим подвыборку

выберем только 4 темы

In [33]:
categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories)
newsgroups_train.filenames.shape

(2034,)

In [34]:
print(newsgroups_train.data[0])
# sample news

From: rych@festival.ed.ac.uk (R Hawkes)
Subject: 3DS: Where did all the texture rules go?
Lines: 21

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Rycharde Hawkes				email: rych@festival.ed.ac.uk
Virtual Environment Laboratory
Dept. of Psychology			Tel  : +44 31 650 3426
Univ. of Edinburgh			Fax  : +44 31 667 0150



In [35]:
newsgroups_train.target[:10]
# у нас 4 класса от 0-3

array([1, 3, 2, 0, 2, 0, 2, 1, 2, 1])

#### TF-IDF(напоминание)

$n_{\mathbb{d}\mathbb{w}}$ - число вхождений слова $\mathbb{w}$ в документ $\mathbb{d}$;<br>
$N_{\mathbb{w}}$ - число документов, содержащих $\mathbb{w}$;<br>
$N$ - число документов; <br><br>

$p(\mathbb{w}, \mathbb{d}) = N_{\mathbb{w}} / N$ - вероятность наличия слова $\mathbb{w}$ в любом документе $\mathbb{d}$
<br>
$P(\mathbb{w}, \mathbb{d}, n_{\mathbb{d}\mathbb{w}}) = (N_{\mathbb{w}} / N)^{n_{\mathbb{d}\mathbb{w}}}$ - вероятность встретить $n_{\mathbb{d}\mathbb{w}}$ раз слово $\mathbb{w}$ в документе $\mathbb{d}$<br><br>

$-\log{P(\mathbb{w}, \mathbb{d}, n_{\mathbb{d}\mathbb{w}})} = n_{\mathbb{d}\mathbb{w}} \cdot \log{(N / N_{\mathbb{w}})} = TF(\mathbb{w}, \mathbb{d}) \cdot IDF(\mathbb{w})$<br><br>

$TF(\mathbb{w}, \mathbb{d}) = n_{\mathbb{d}\mathbb{w}}$ - term frequency;<br>
$IDF(\mathbb{w}) = \log{(N /N_{\mathbb{w}})}$ - inverted document frequency;

чем больше слово встречается в документе, тем вероятность больше стремится к 0. а чем борльше вероятность, тем важнее документ

чем метрика tf-idf больше, тем важнее для документа это слово

TF - количество вхождений


#### Давайте векторизуем эти тексты с помощью TF-IDF

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### Некоторые параметры:
* input : string {‘filename’, ‘file’, ‘content’}
*  lowercase : boolean, default True
*  preprocessor : callable or None (default)
*  tokenizer : callable or None (default)
*  stop_words : string {‘english’}, list, or None (default)
*  ngram_range : tuple (min_n, max_n)
*  max_df : float in range [0.0, 1.0] or int, default=1.0
*  min_df : float in range [0.0, 1.0] or int, default=1
*  max_features : int or None, default=None

#### Перебор параметров

In [37]:
# lowercase 
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

(2034, 34118)

In [38]:
# приведем все к нижнему регистру, количество слов отличается на 8000. будем использовать lowecase
vectorizer = TfidfVectorizer(lowercase=False)
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

(2034, 42307)

In [39]:
vectorizer.get_feature_names_out()[:10]
#посмотрим какие слова в нашем словаре и видим. что они неподходящие

array(['00', '000', '0000', '00000', '000000', '000005102000', '000021',
       '000062David42', '0000VEC', '0001'], dtype=object)

In [40]:
# min_df, max_df посмотрим слова, которые встречаются больше, чем в 80% случаев
vectorizer = TfidfVectorizer(min_df=0.8)
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

(2034, 9)

In [41]:
vectorizer.get_feature_names_out()

array(['and', 'from', 'in', 'lines', 'of', 'organization', 'subject',
       'the', 'to'], dtype=object)

нам эти слова не нужна, исключим их

In [42]:
vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.8)
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

(2034, 2391)

из 42307 получили 2391

In [43]:
# ngram_range будем смотреть не только частоту появления слов, но и их взаимодейсвие между собой с помощью n-грамм
vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=0.03, max_df=0.9)
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

(2034, 1236)

In [44]:
# стоп-слова, preproc будем наши стор-слова закижывать в препроцесс
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
# используем лематайзер wordnet
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()
# будем избавляться от наших стоп-слов и леммитизировать по-отдельности
def preproc_nltk(text):
    #text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stopWords])

st = "Oh, I think I ve landed Where there are miracles at work,  For the thirst and for the hunger Come the conference of birds"
preproc_nltk(st)

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'oh , think landed miracle work , thirst hunger come conference bird'

In [45]:
#посмотрим время работы векторайзера на preproc nltk
%time
vectorizer = TfidfVectorizer(preprocessor=preproc_nltk)
vectors = vectorizer.fit_transform(newsgroups_train.data)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


In [46]:
# preproc_spacy используем spacy/ он заменяет стоп-слова на название члена предложения. 
!pip install -U -q spacy
!python -m -q spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
texts = newsgroups_train.data.copy()

def preproc_spacy(text):
    spacy_results = nlp(text)
    return ' '.join([token.lemma_ for token in spacy_results if token.lemma_ not in stopWords])
preproc_spacy(st)

/home/codespace/.python/current/bin/python: No module named -q


'oh , I think I land miracle work ,   thirst hunger come conference bird'

In [47]:
# spacy делаает еще задачу parser и ner/ но было бы неплохо как-то отделять. рекомендуют работать с помощью некоторого пайплайна работы с текстом
# берем леммы и убираем стоп-слова
%time
new_texts = []
for doc in nlp.pipe(texts, batch_size=32, n_process=3, disable=["parser", "ner"]):
    new_texts.append(' '.join([tok.lemma_ for tok in doc if tok.lemma not in stopWords]))
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(new_texts)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 7.39 µs


In [48]:
print(new_texts[0])

from : rych@festival.ed.ac.uk ( R Hawkes ) 
 subject : 3ds : where do all the texture rule go ? 
 line : 21 

 Hi , 

 I have notice that if you only save a model ( with all your mapping plane 
 position carefully ) to a .3ds file that when you reload it after restart 
 3ds , they be give a default position and orientation .   but if you save 
 to a .prj file their position / orientation be preserve .   do anyone 
 know why this information be not store in the .3ds file ?   nothing be 
 explicitly say in the manual about save texture rule in the .prj file . 
 I would like to be able to read the texture rule information , do anyone have 
 the format for the .PRJ file ? 

 be the .cel file format available from somewhere ? 

 rych 

 = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
 Rycharde Hawkes 				 email : rych@festival.ed.ac.uk 
 Virtual Environment Laboratory 
 Dept . of psychology 			 Tel 

качество лемитизации лучше у spacy? чем nltk. но разница во времени

#### Итоговая модель

In [75]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.5, max_features=1000)
# max_features=1000 размер вектора ограничиваем 1000
vectors = vectorizer.fit_transform(new_texts)
vectorizer.get_feature_names_out()[::100]

array(['000', 'attempt', 'choose', 'engineering', 'human', 'look',
       'of this', 'report', 'technology', 'understand'], dtype=object)

теперь каждый текст замениои на некий вектор. 
вектор размера нашего словаря и внутри него находятся значения tfidf слов

#### Можем посмотреть на косинусную меру между векторами

In [78]:
vectors.shape
print(vectors.get_shape)
# max_features=1000 размер вектора ограничили

<bound method spmatrix.get_shape of <2034x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 182658 stored elements in Compressed Sparse Row format>>


In [79]:
# выведем ненулевые векторы. 50 из 1000. это плохо
vector = vectors.toarray()[0]
(vector != 0).sum()

52

In [56]:
import numpy as np
from numpy.linalg import norm
# используем матрицу sparse для экономии места
type(vectors)

scipy.sparse._csr.csr_matrix

In [57]:
np.mean(list(map(lambda x: (x != 0).sum(), vectors.todense())))

89.8023598820059

In [84]:
dense_vectors = vectors.todense()
dense_vectors.shape

(2034, 1000)

но мы будем использовать numpy маьрицу вместо sparse

отличие косинусного расстояния и косинусного мер близости - кос расстояние - это 1 - косинусная схождесть. тем меньше тем лучше. у близости тем больше, тем лучше

In [85]:
def cosine_sim(v1, v2):
    # v1, v2 (1 x dim)
    return np.asarray(v1 @ v2.T / norm(v1) / norm(v2))[0][0]

In [86]:
cosine_sim(dense_vectors[0], dense_vectors[0])

1.0000000000000002

In [87]:
#подсчитаем косинусное расстояние между 10 первыми статьями
cosines = []
for i in range(10):
    cosines.append(cosine_sim(dense_vectors[0], dense_vectors[i]))

In [72]:
# [1, 3, 2, 0, 2, 0, 2, 1, 2, 1] метки классов
# видим максимальная схожесть с самим собой 1.0000\ с 8 новостью максимальная ит эта класс номер 1\ можем по косиносному расстоянию понимать насколько новости связаны между собой
cosines

[1.0000000000000002,
 0.04191279776414236,
 0.00586838361101993,
 0.09771238093526102,
 0.0706091645327028,
 0.06745764842966309,
 0.026714182362747592,
 0.22853760897260955,
 0.03163642012466396,
 0.06928662593161493]

#### Обучим любую известную модель на полученных признаках

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import SGDClassifier
# разбили данные на тестовую и выборку для обучения 

X_train, X_test, y_train, y_test= train_test_split(dense_vectors, newsgroups_train.target, test_size=0.2, random_state=0)
y_train.shape, y_test.shape

((1627,), (407,))

In [74]:
#обучим с помощью svm - метод опорных векторов  для задачи классификации
%time
svc = svm.SVC()
svc.fit(X_train, y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


TypeError: np.matrix is not supported. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

In [66]:
accuracy_score(y_test, svc.predict(X_test))

NotFittedError: This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
accuracy_score(y_test, sgd.predict(X_test))

0.8968058968058968

### Embeddings

In [ ]:
import gensim.downloader as api
embeddings_pretrained = api.load('glove-twitter-25')

In [ ]:
from gensim.models import Word2Vec

proc_words = [preproc_nltk(text).split() for text in newsgroups_train.data]
embeddings_trained = Word2Vec(proc_words, # data for model to train on
                 size=100,                 # embedding vector size
                 min_count=3,             # consider words that occured at least 5 times
                 window=3).wv

In [ ]:
def vectorize_sum(comment, embeddings):
    """
    implement a function that converts preprocessed comment to a sum of token vectors
    """
    embedding_dim = embeddings.vectors.shape[1]
    features = np.zeros([embedding_dim], dtype='float32')

    for word in preproc_nltk(comment).split():
        if word in embeddings:
            features += embeddings[f'{word}']

    return features

In [ ]:
len(embeddings_trained.index2word)

13651

In [ ]:
X_wv = np.stack([vectorize_sum(text, embeddings_pretrained) for text in newsgroups_train.data])
X_train_wv, X_test_wv, y_train, y_test = train_test_split(X_wv, newsgroups_train.target, test_size=0.2, random_state=0)
X_train_wv.shape, X_test_wv.shape

((1627, 25), (407, 25))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

clf = LogisticRegression(max_iter=5000)
wv_model = clf.fit(X_train_wv, y_train)
accuracy_score(y_test, wv_model.predict(X_test_wv))

0.7100737100737101

In [ ]:
X_wv = np.stack([vectorize_sum(text, embeddings_trained) for text in newsgroups_train.data])
X_train_wv, X_test_wv, y_train, y_test = train_test_split(X_wv, newsgroups_train.target, test_size=0.2, random_state=0)
X_train_wv.shape, X_test_wv.shape

((1627, 100), (407, 100))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

clf = LogisticRegression(max_iter=10000)
wv_model = clf.fit(X_train_wv, y_train)
accuracy_score(y_test, wv_model.predict(X_test_wv))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8402948402948403